In [ ]:
import tensorflow as tf

In [ ]:
def ResBlock(inputs, filters=256):
    x = tf.keras.layers.Conv2D(filters=filters, kernel_size=3, strides=1, 
                               padding="same", kernel_initializer='he_normal', activation="relu")(inputs)
    x = tf.keras.layers.Conv2D(filters=filters, kernel_size=3, strides=1, 
                               padding="same", kernel_initializer='he_normal')(x)
    x = tf.keras.layers.Lambda(lambda t: t * 0.1)(x)
    x = tf.keras.layers.Add()([inputs, x])
    return x

def Upsampling(inputs, filters=256, factor=2):
    x = tf.keras.layers.Conv2D(filters=filters*(factor**2), kernel_size=3, 
                               padding="same", kernel_initializer='he_normal')(inputs)
    x = tf.nn.depth_to_space(x, block_size=factor)
    return x

def PSNR(sr, hr):
    psnr_value = tf.reduce_mean(tf.image.psnr(hr, sr, max_val=255))
    return psnr_value

In [ ]:
def model():
    
    inputs = tf.keras.layers.Input(shape=(None, None, 3))
    rescale = tf.keras.layers.Rescaling(scale=1.0 / 255)(inputs)
    
    x = r = tf.keras.layers.Conv2D(filters=256, kernel_size=3, strides=1,
                                       padding="same", kernel_initializer='he_normal')(rescale)
    
    for _ in range(32):
        r = ResBlock(r)
    
    r = tf.keras.layers.Conv2D(filters=256, kernel_size=3, strides=1,
                               padding="same", kernel_initializer='he_normal')(r)
    x = x + r

    x = Upsampling(x)
    x = Upsampling(x)
    
    x = tf.keras.layers.Conv2D(filters=3, kernel_size=3, strides=1,
                               padding="same", kernel_initializer='he_normal')(x)
    x = tf.keras.layers.Rescaling(scale=255)(x)
    return tf.keras.Model(inputs = inputs, outputs = x)

In [ ]:
model=model()
model.summary()